The purpose of this script is to tie together the input generator and neural model scripts to train a neural network.

In [1]:
# Import the desired functions from the input generator and neural model scripts:
from imdb_input_generator import *
from imdb_neural_model import *

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Import other required libraries:
import pickle as pkl
import h5py
import os
import pandas as pd

In [19]:
# Import Keras:
import keras

In [20]:
# Function for checking the latest training epoch number:
def epoch_check(count, path, name_template, ext):
    for epoch in os.listdir(path):
        e = epoch.split(name_template)
        e = e[1].split(ext)
        e = int(e[0])
        if e > count:
            count = e
    return count

In [5]:
# Specify directory containing data files:
parent_path = '/Users/cheng-haotai/Documents/Projects_Data/IMDb_Predictor/'
data_name = 'training_data/'
data_path = parent_path + data_name

In [6]:
# Specify training data files that have been saved:
train1_name = 'train1_index.pickle'
train2_name = 'train2_index.pickle'
train3_name = 'train3_index.pickle'
train1_path = data_path + train1_name
train2_path = data_path + train2_name
train3_path = data_path + train3_name

In [7]:
# Specify input data file that has been saved:
input_vector_name = 'input_vectors.h5'
input_vector_path = data_path + input_vector_name

In [8]:
# Specify output data file that has been saved:
output_vector_name = 'output_data.pickle'
output_vector_path = data_path + output_vector_name

In [9]:
# Specify test index data files that have been saved:
test_name = 'test_index.pickle'
test_name_path = data_path + test_name

In [16]:
# Specify training weight file details for auto-encoder:
weight_folder = data_path + 'autoen_training_weights/'
weight_name_template = 'autoen_weights'
weight_name_ext = '.h5'

In [17]:
# Specify training weight file details for imdb regression model:
imdb_weight_folder = data_path + 'imdb_training_data/'
imdb_weight_name_template = 'imdb_weights'
imdb_weight_name_ext = '.h5'

In [18]:
# Prepare inputs for imdb_regression model:
transformed_data_name = 'transformed_data.h5'
transformed_data_path = data_path + transformed_data_name
transformed_data_dict = 'transformed_data'

In [21]:
# Specify save parameters for predictions:
prediction_data_name = 'score_predictions.csv'
prediction_data_path = data_path + prediction_data_name

The following code will be partitioned by function. These functions can be turned on / off by modifying the cell below to "1" or "0".

In [10]:
# Setting flags to turn on/off training 
regression = 1
ae = 0
test_ae = 0  # Allows for early termination of autoencoder training process
test_regression = 1

In [ ]:
if ae == 1:
    # Auto-Encoder Portion:
    print('Starting auto-encoder portion:')
    # Get size of network input:
    with h5py.File(input_vector_path, 'r') as input_file:
        input_data = input_file['input_dataset'][:]
    
    model_len = len(input_data)
    row_len = len(input_data[0])
    model_size = (model_len,)  # Tuple of size 1
    row_size = (row_len,)
    print('Tuple to input into auto encoder function has size:', model_size)
    # Create a model object of the auto-encoder:
    print('Generating model for encoder and auto-encoder:')
    encoder, autoencoder = auto_encoder(model_size)
    
    # Check if weight folder exists and create if it doesn't:
    # Typically all training weights for a model will live in one folder
    if not os.path.exists(weight_folder):
        print('Creating folder for auto-encoder weights...')
        os.mkdir(weight_folder)
    else:
        print('Folder for auto-encoder weights already exists!')
        
    # If the weight folder exists, check how many epochs have been processed:
    counter = -1
    if os.path.exists(weight_folder):
        # Finding the number assigned to most recent epoch
        counter = epoch_check(counter, weight_folder, weight_name_template, weight_name_ext)
        print 'Most up-to-date auto-encoder weight file (epoch) is indexed:', counter
    else:
        # Set counter to -1 if no epochs have been processed
        counter = -1
        print('No epochs for auto-encoder have run yet')
    
    # Check if the latest weight file can be loaded:
    weight_file_name = weight_folder + weight_name_template + str(counter) + weight_name_ext
    if os.path.exists(weight_file_name):
        print('Latest auto-encoder weight file already exists! Loading...')
        # by_name allows for old weight files to be used with new models with new structures
        autoencoder.load_weights(weight_file_name, by_name=True)
    else:
        print('No auto-encoder weights have been generated yet')
    
    if test_ae == 0:
        print 'Generating weights from latest weight file...'
        # Specify maximum number of epochs to process:
        epoch_num = 1000
        for num in range(counter + 1, epoch_num):
            print('Auto-encoder weight generating on epoch', num)
            # Train (implies updating error) and generate weights:
            # nb_epoch specifies how many epochs run before saving (1 in this case)
            # samples_per_epoch specifies # of times to call generator
            autoencoder.fit_generator(autoencoder_generator(train1_path, input_vector_path), samples_per_epoch=model_len, nb_epoch=1)
            fresh_weight_name = weight_folder + weight_name_template + str(num) + weight_name_ext
            autoencoder.save_weights(fresh_weight_name)
    else:
        print 'No further weight generation required'
        # Assuming that all epochs have been processed and no need to check last epoch:
        # Each weight file is the next progression (weight update) of the previous weight file
        # Loading weights trained from autoencoder (shared weights)
        # Encoder can "see" weights because by_name = True
        print('Loading latest weight file into encoder model...')
        latest_weight = weight_folder + weight_name_template + str(counter) + weight_name_ext
        encoder.load_weights(latest_weight, by_name=True)

        # Use auto_encoder to encode data into small dimension (utilizing encoder layer):
        if not os.path.exists(transformed_data_path):
            print('Transforming input data into lower dimensionality...')
            transformed_data = []
            for row in input_data:
                placeholder_input = np.zeros((1, row_len))  # 22000 size placeholder
                placeholder_input[0] = row
                # Gives result of encoder layer in auto_encoder function
                # Predict doesn't update error - simply multiplies input by weights
                en_predict = encoder.predict(placeholder_input)[0]  # list with 1 list in it & needed to take the first element
                transformed_data.append(en_predict)
            # Save transformed inputs into h5 file:
            print('Saving transformed data into h5py format...')
            transformed_data_file = h5py.File(transformed_data_path, 'w')  # Aaron: wb turned to w
            transformed_data_file.create_dataset(transformed_data_dict, data=transformed_data)
        else:
            print('Data has already been transformed in dimensionality!')


In [ ]:
if regression == 1:
    # Post Auto-Encoder Portion:
    print 'Progressing onto regression model...'

    print 'Loading transformed data...'
    with h5py.File(transformed_data_path, 'r') as load_transformed:
        transformed_data = load_transformed['transformed_data'][:]

    # Find length of input data from transformed data:
    transformed_len = len(transformed_data)
    trans_row_len = len(transformed_data[0])
    trans_row_size = (trans_row_len,)  # Length 1 tuple (10000 value)
    # Create model object of imdb regression model:
    imdb_reg = imdb_regression(transformed_size)
    
    # Check if weight folder exists and create if it doesn't:
    # Typically all training weights for a model will live in one folder
    if not os.path.exists(imdb_weight_folder):
        os.mkdir(imdb_weight_folder)

    # If the weight folder exists, check how many epochs have been processed:
    imdb_counter = -1
    if os.path.exists(imdb_weight_folder):
        # Finding the number assigned to most recent epoch
        imdb_counter = epoch_check(imdb_counter, imdb_weight_folder, imdb_weight_name_template, imdb_weight_name_ext)
        print 'Latest IMDb regression weight file (epoch) is indexed', imdb_counter
    else:
        imdb_counter = -1
    
    # Define name of imdb weight file:
    imdb_weight_file_name = imdb_weight_folder + imdb_weight_name_template + str(imdb_counter) + imdb_weight_name_ext
    # Load imdb weight file if it exists:
    if os.path.exists(imdb_weight_file_name):
        print 'Latest imdb weight file already exists! Loading...'
        # by_name allows for old weight files to be used with new models with new structures
        imdb_reg.load_weights(imdb_weight_file_name, by_name=True)
    else:
        print 'No imdb weights have been generated yet'
    
    if test_regression == 0:
        # Specify maximum number of epochs to process in imdb weight training:
        imdb_epoch_num = 1000
        for num in range(imdb_counter + 1, imdb_epoch_num):
            print('IMDb regression weight training on epoch:', num)
            imdb_reg.fit_generator(input_generator(train1_path, output_vector_path, transformed_data_path),
                                   samples_per_epoch = transformed_len,
                                   nb_epoch=1)
            fresh_imdb_weight_name = imdb_weight_folder + imdb_weight_name_template + str(num) + imdb_weight_name_ext
            imdb_reg.save_weights(fresh_imdb_weight_name)
            
    else:
        # Load latest weights for imdb model:
        imdb_latest_weight = imdb_weight_folder + imdb_weight_name_template + str(imdb_counter) + imdb_weight_name_ext
        imdb_reg.load_weights(imdb_latest_weight, by_name=True)

        # Run predictions on test data:
        # Load test index file and output data file:
        with open(test_name_path, 'rb') as test_set:
            test_data = pkl.load(test_set)
        with open(output_vector_path, 'rb') as output_set:
            output_data = pkl.load(output_set)

        # Check whether or not predictions have been run:
        if not os.path.exists(prediction_data_path):
            print('Running predictions on test data...')
            prediction_vec = []
            score_vec = []
            # Train on test data:
            for index in test_data:
                # Pull necessary data using index in test data:
                test_row = transformed_data[index]
                score_val = output_data[index]
                # Populate placeholder vector:
                placeholder_input = np.zeros((1, trans_row_len))  # 10000 placeholder
                placeholder_input[0] = test_row
                imdb_predict = imdb_reg.predict(placeholder_input)
                prediction_vec.append(imdb_predict)
                score_vec.append(score_val)
            
            print('Predictions have completed! Proceeding to save data...')
            final_results = pd.DataFrame()
            final_results['Real_Score'] = score_vec
            final_results['Predicted_Score'] = prediction_vec
            final_results.to_csv(prediction_data_path)
        else:
            print('Predictions for IMDb scores have already been run!')
            # final_results = pd.read_csv(prediction_data_path)